In [1]:
import numpy as np 
import pandas as pd
import cv2
import glob
import re
from pathlib import Path

### Convert DLC detections to YOLO Detection format

Format :
.csv : 
Sr. No, Frame, Pos x, Pos y , Territory_id


MOT Format : 
FIle name , frame , track id, bb_left, bb_top, bb_width, bb_height, confidence, class id, class_name 

In [2]:
DATE = ['20230318']#, '20230303']
SESSION = ['SM_Lek1']#, 'SE_Lek1']
DRONE = ['P3D5']#, 'P1D2', 'P2D3', 'P2D4', 'P3D5', 'P3D6']

file_directory = '/Volumes/SSD5'
box_size = 80

In [3]:
for date in DATE:
    for session in SESSION:
        for drone in DRONE:
            internal_path = Path(file_directory, date, session, drone)
            files = list(internal_path.rglob(f"{date}_{session}_{drone}_DJI_0[0-9][0-9][0-9].csv"))
            print(f"{date}-{session}-{drone}: {len(files)} matches")
            
            for file in files:
                print(file)
                df_dlc = pd.read_csv(file)

                # Create an empty dataframe with headers in YOLO detection format
                headers = ['file_path', 'frame', 'track_id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'confidence', 'class_id', 'class_name']
                df = pd.DataFrame(columns=headers).reset_index(drop=True)

                df['frame'] = df_dlc['frame'].astype(int)
                df[['bb_left', 'bb_top']] = df_dlc[['pos_x', 'pos_y']]
                df['file_path'] = internal_path
                df['confidence'] = 0.5
                df['class_id'] = 5
                df['class_name'] = 'territory'
                df['track_id'] = -1

                # DLC models were trained on a third of the video size
                df['bb_left'] *= 3
                df['bb_top'] *= 3

                # Convert keypoint to bounding box
                df['bb_left'] -= box_size/2
                df['bb_top'] -= box_size/2
                df['bb_width'] = box_size
                df['bb_height'] = box_size

                # Save modified file
                df.to_csv(str(file)[:-4] + '_YOLO.csv', index=False)

20230318-SM_Lek1-P3D5: 12 matches
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0690.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0691.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0698.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0699.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0700.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0701.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0702.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0705.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0706.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0707.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0708.csv
/Volumes/SSD5/20230318/SM_Lek1/P3D5/20230318_SM_Lek1_P3D5_DJI_0709.csv
